In [31]:
from self_supervised_halos.utils.utils import data_postprocess_path, check_cuda
from self_supervised_halos.scripts.models import Classification_2d
from self_supervised_halos.utils.dataloader import HaloDataset, img2d_transform, subhalos_df, DataLoader

import torch
import torch.nn as nn
import torch.optim as optim



device = check_cuda()

CUDA is not available.
Device: cpu


In [32]:
batch_size = 32

dataset = HaloDataset(root_dir=data_postprocess_path,subhalos_df=subhalos_df, 
                      load_2d=True, load_3d=False, load_mass=False,
                        choose_two_2d = False,
                      DEBUG_LIMIT_FILES = 500)

if device=='cpu':
    batch_size = 128
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
else:
    print('dataloader on gpu')
    batch_size = 512
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers = 2)


print(f'Batch size: {batch_size}')

n_data = len(dataset)
f_train = 0.6
f_val = 0.2
f_test = 1 - f_train - f_val


train_size = int(f_train*len(dataset))
val_size = int(f_val*len(dataset))
test_size = len(dataset) - train_size - val_size


trainval_ds, test_ds = torch.utils.data.random_split(dataset, [train_size+val_size, test_size])
train_size = int(f_train/(f_train+f_val)*len(trainval_ds))
val_size = len(trainval_ds) - train_size

train_ds, val_ds = torch.utils.data.random_split(trainval_ds, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=True)



Preparing 2D data: 100%|██████████| 500/500 [00:01<00:00, 440.60it/s]

Batch size: 128


In [33]:
lr = 1e-3
criterion = nn.CrossEntropyLoss(weight=dataset.mass_bins_weights.to(device)).to(device)


model = Classification_2d(
                        optimizer_class= optim.Adam,
                        optimizer_params = dict(lr=lr),
                        scheduler_class= optim.lr_scheduler.ReduceLROnPlateau,
                        scheduler_params = dict(mode='min', factor=0.5, patience=5, verbose=True),
                        history = None,
                        device = device,
                        augmentations = img2d_transform)



AttributeError: cannot assign module before Module.__init__() call

In [ ]:
classification_2d = Classification_2d(
                        